In [1]:
from ns3gym import ns3env
from comet_ml import Experiment
import tqdm
import subprocess
from collections import deque
import numpy as np

from agents.ddpg.agent import Agent, Config
# from agents.dqn.agent import Agent, Config
# from agents.dqn.model import QNetworkTf
from agents.teacher import Teacher

C:\ProgramData\Anaconda3\lib\site-packages\requests\__init__.py:80: RequestsDependencyWarning: urllib3 (1.24.1) or chardet (3.0.4) doesn't match a supported version!
  RequestsDependencyWarning)
C:\ProgramData\Anaconda3\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


### Basic constants and setting up environment

In [2]:
# config = Config(buffer_size=2e3, batch_size=64, gamma=0.99, tau=1e-3, lr=5e-4)
config = Config(buffer_size=2.6e3, batch_size=64, gamma=0.99, tau=1e-3, lr_actor=1e-5, lr_critic=4e-3)

In [3]:
scenario = "convergence"

simTime = 10 # seconds
stepTime = 0.1  # seconds
history_length = 10


EPISODE_COUNT = 30
steps_per_ep = int(simTime/stepTime)

SCRIPT_RUNNING = False

script_exec_command = f'../../waf --run "linear-mesh --simTime={simTime} --envStepTime={stepTime} --historyLength={history_length} --agentType={Agent.TYPE} --nonZeroStart --scenario={scenario} --nWifi=7"'
print("Steps per episode:", steps_per_ep)

Steps per episode: 100


In [ ]:
port = 5555
seed = 0

simArgs = {"--simTime": simTime,
           "--testArg": 123,
           "--nodeNum": 5,
           "--distance": 500}
debug = False

subprocess.Popen(['bash', '-c', script_exec_command])
SCRIPT_RUNNING = True

env = ns3env.Ns3Env(port=port, stepTime=stepTime, startSim=0, simSeed=seed, simArgs=simArgs, debug=debug)

Waiting for simulation script to connect on port: tcp://localhost:5555
Please start proper ns-3 simulation script using ./waf --run "..."


In [ ]:
env.reset()
ob_space = env.observation_space
ac_space = env.action_space

print("Observation space shape:", ob_space)
print("Action space shape:", ac_space)

assert ob_space is not None

### Creating and training agent

In [ ]:
# import tensorflow as tf

# class Network(QNetworkTf):
#     def _inference(self):
#         with tf.variable_scope("inference_"+self.name):
#             layer = tf.layers.dense(self.input, 1024, activation=tf.nn.relu)
#             layer = tf.layers.dense(layer, 512, activation=tf.nn.relu)
# #             layer = tf.layers.batch_normalization(layer)
#             layer = tf.layers.dense(layer, 512, activation=tf.nn.relu)
#             layer = tf.layers.dense(layer, 256, activation=tf.nn.relu)
#             layer = tf.layers.dense(layer, 256, activation=tf.nn.relu)
#             layer = tf.layers.dense(layer, 64, activation=tf.nn.relu)
#             layer = tf.layers.dense(layer, self.action_size)
#         return layer

In [ ]:
if not SCRIPT_RUNNING:
    subprocess.Popen(['bash', '-c', script_exec_command])
    SCRIPT_RUNNING = True

agent = Agent(history_length, action_size=1, config=config)
# agent.set_epsilon(0.9, 0.01, 25)
teacher = Teacher(agent, env, 1)
teacher.train(EPISODE_COUNT, simTime, stepTime, script_exec_command, "Inp: window Mb sent", "Rew: normalized speed", "Warmup", "Reward - 1/2", "DDPG", "Convergence", "Action offset", "Reward offset")

SCRIPT_RUNNING = False

In [ ]:
teacher.last_actions